In [ ]:
import pandas as pd
import numpy as np
from functools import reduce

In [3]:
# read the files
no_DNF_results_data_file = pd.read_csv(r"/home/winter-storm/f1-data-project/erdos_ds_f1/cleaned_data/f1db-races-race-results-NO-DNF.csv")
quals_data_file = pd.read_csv(r"/home/winter-storm/f1-data-project/erdos_ds_f1/cleaned_data/f1db-races-qualifying-results-CLEANED.csv")
free_practice_1_data = pd.read_csv(r"/home/winter-storm/f1-data-project/erdos_ds_f1/cleaned_data/f1db-races-free-practice-1-results-CLEANED.csv")
free_practice_2_data = pd.read_csv(r"/home/winter-storm/f1-data-project/erdos_ds_f1/cleaned_data/f1db-races-free-practice-2-results-CLEANED.csv")
free_practice_3_data = pd.read_csv(r"/home/winter-storm/f1-data-project/erdos_ds_f1/cleaned_data/f1db-races-free-practice-3-results-CLEANED.csv")
free_practice_4_data = pd.read_csv(r"/home/winter-storm/f1-data-project/erdos_ds_f1/cleaned_data/f1db-races-free-practice-4-results-CLEANED.csv")
start_position_data_file = pd.read_csv(r"/home/winter-storm/f1-data-project/erdos_ds_f1/data_f1db/f1db-races-starting-grid-positions.csv")
start_position_data_file = start_position_data_file[
    (start_position_data_file['year'] > 1985) & (start_position_data_file['year'] < 2025)
    ]

# Create list of unique race ids
race_id_list = no_DNF_results_data_file['raceId'].unique()

In [4]:
# Compute the Average Local Position Difference (ALPS) between Fastest Qualifying Time grid and the Starting Grid

quals_data_file['minQualTime'] = quals_data_file[['timeMillis', 'q1Millis', 'q2Millis', 'q3Millis']].min(axis=1)

def generate_quals_ALPC_data(input_race_id_list):
    quals_avg_local_position = []

    for race_id in input_race_id_list:

        this_race_data = quals_data_file[quals_data_file['raceId'] == race_id]

        if this_race_data.isna().all(axis=1).all():
            quals_avg_local_position.append({'raceId': race_id, 'qualifyingALPC': np.nan,
                                              'qualAvgPosDiff': np.nan})

        this_race_data = this_race_data.sort_values('minQualTime')
        this_race_start_data = start_position_data_file[start_position_data_file['raceId'] == race_id]
        this_race_start_data = this_race_start_data.sort_values('positionNumber')
        min_qual_time_list = this_race_data['driverId'].to_list()
        starting_grid = this_race_start_data['driverId'].to_list()

        n = len(starting_grid)
        m = len(min_qual_time_list)
        this_sum = 0
        position_diff = []

        if m != n: 
            common_drivers = set(min_qual_time_list) & set(starting_grid)
            min_qual_time_list = [x for x in min_qual_time_list if x in common_drivers]
            starting_grid = [y for y in starting_grid if y in common_drivers]
            n = len(starting_grid)

        for i in range(n):
            summand = 0
            for j in range(n):
                if j == i: continue
                driver_i = starting_grid[i]
                driver_j = starting_grid[j]
                summand += (i-j) - (min_qual_time_list.index(driver_i)-min_qual_time_list.index(driver_j))
            this_sum += abs(summand)

        for i in range(n):
            driver_i = starting_grid[i]
            position_diff.append(abs(i - min_qual_time_list.index(driver_i)))

        this_sum = this_sum/(n * (n-1))
        quals_avg_local_position.append({'raceId': race_id, 'qualifyingALPC': this_sum, 
                                         'qualAvgPosDiff': np.mean(position_diff)})

    return pd.DataFrame(quals_avg_local_position)
generate_quals_ALPC_data(race_id_list)

,raceId,qualifyingALPC,qualAvgPosDiff
0,421,0.000000,0.000000
1,422,0.000000,0.000000
2,423,0.000000,0.000000
3,424,0.000000,0.000000
4,425,0.000000,0.000000
...,...,...,...
700,1121,0.105263,0.100000
701,1122,1.294118,1.222222
702,1123,1.368421,1.300000
703,1124,0.210526,0.200000


In [5]:
fpdf = free_practice_1_data[['raceId', 'driverId', 'timeMillis']].rename(columns={'timeMillis': 'fp1Time'})

fpdf = fpdf.merge(
    free_practice_2_data[['raceId', 'driverId', 'timeMillis']].rename(columns={'timeMillis': 'fp2Time'}),
    on=['raceId', 'driverId'],
    how='left'
)

fpdf = fpdf.merge(
    free_practice_3_data[['raceId', 'driverId', 'timeMillis']].rename(columns={'timeMillis': 'fp3Time'}),
    on=['raceId', 'driverId'],
    how='left'
)

fpdf['min_fp_time'] = fpdf[['fp1Time', 'fp2Time', 'fp3Time']].min(axis=1)
fpdf['min_fp_time'] = fpdf['min_fp_time'].fillna(1000000)
fpdf

,raceId,driverId,fp1Time,fp2Time,fp3Time,min_fp_time
0,435,nelson-piquet,78601.0,78353.0,NaN,78353.0
1,435,ayrton-senna,78779.0,77977.0,NaN,77977.0
2,435,gerhard-berger,79004.0,78088.0,NaN,78088.0
3,435,keke-rosberg,79099.0,79160.0,NaN,79099.0
4,435,nigel-mansell,79588.0,78785.0,NaN,78785.0
...,...,...,...,...,...,...
14229,1125,fernando-alonso,85504.0,84574.0,84453.0,84453.0
14230,1125,liam-lawson,85563.0,84503.0,84519.0,84503.0
14231,1125,valtteri-bottas,85611.0,84230.0,84479.0,84230.0
14232,1125,guanyu-zhou,85921.0,84557.0,84668.0,84557.0


In [6]:
fp_race_id_list = fpdf['raceId'].unique()

def get_free_practice_RALPC(input_race_id_list):
    fp_ralpc = []

    for race_id in input_race_id_list:
        if race_id not in fpdf['raceId'].unique():
            fp_ralpc.append({'raceId': race_id, 'FRALPC': np.nan, 'freeAvgPosDiff': np.nan})
            continue

        this_race_data = fpdf[fpdf['raceId'] == race_id]
        this_race_start_data = start_position_data_file[start_position_data_file['raceId'] == race_id]
        this_race_start_data = this_race_start_data.sort_values('positionNumber')
        update_race_df = this_race_data.sort_values('min_fp_time')
        driver_fp_pos = update_race_df['driverId'].to_list()
        starting_grid = this_race_start_data['driverId'].to_list()

        n = len(starting_grid)
        m = len(driver_fp_pos)
        total_sum = 0
        pos_diff = []

        if m != n: 
            common_drivers = set(driver_fp_pos) & set(starting_grid)
            driver_fp_pos = [x for x in driver_fp_pos if x in common_drivers]
            starting_grid = [y for y in starting_grid if y in common_drivers]
            n = len(starting_grid)

        for i in range(n):
            summand = 0
            for j in range(n):
                if i == j: continue
                driver_i = starting_grid[i]
                driver_j = starting_grid[j]
                local_change = (i - j) - (driver_fp_pos.index(driver_i) - driver_fp_pos.index(driver_j))
                summand += local_change

            total_sum += abs(summand) / (n*(n-1))

        for i in range(n):
            driver_i = starting_grid[i]
            pos_diff.append(abs(i - driver_fp_pos.index(driver_i)))
        
        fp_ralpc.append({'raceId': race_id, 'FRALPC': total_sum, 'freeAvgPosDiff': np.mean(pos_diff)})
    return pd.DataFrame(fp_ralpc) 

get_free_practice_RALPC(race_id_list)

,raceId,FRALPC,freeAvgPosDiff
0,421,NaN,NaN
1,422,NaN,NaN
2,423,NaN,NaN
3,424,NaN,NaN
4,425,NaN,NaN
...,...,...,...
700,1121,3.714286,3.466667
701,1122,4.705882,4.444444
702,1123,4.315789,4.100000
703,1124,5.052632,4.800000


In [7]:
pacedf = free_practice_1_data[['raceId', 'driverId', 'timeMillis']].rename(columns={'timeMillis': 'fp1Time'})

pacedf = pacedf.merge(
    free_practice_2_data[['raceId', 'driverId', 'timeMillis']].rename(columns={'timeMillis': 'fp2Time'}),
    on=['raceId', 'driverId'],
    how='left'
)

pacedf = pacedf.merge(
    free_practice_3_data[['raceId', 'driverId', 'timeMillis']].rename(columns={'timeMillis': 'fp3Time'}),
    on=['raceId', 'driverId'],
    how='left'
)

pacedf = pacedf.merge(
    quals_data_file[['raceId', 'driverId', 'timeMillis', 'q1Millis', 'q2Millis', 'q3Millis']].rename(columns={'timeMillis': 'qMillis'}),
    on=['raceId', 'driverId'],
    how='left'
)

pacedf['min_time'] = pacedf[['fp1Time', 'fp2Time', 'fp3Time', 'qMillis', 'q1Millis', 'q2Millis', 'q3Millis']].min(axis=1)
pacedf['min_time'] = pacedf['min_time'].fillna(1000000)
pacedf

,raceId,driverId,fp1Time,fp2Time,fp3Time,qMillis,q1Millis,q2Millis,q3Millis,min_time
0,435,nelson-piquet,78601.0,78353.0,NaN,77279.0,NaN,NaN,NaN,77279.0
1,435,ayrton-senna,78779.0,77977.0,NaN,76990.0,NaN,NaN,NaN,76990.0
2,435,gerhard-berger,79004.0,78088.0,NaN,77609.0,NaN,NaN,NaN,77609.0
3,435,keke-rosberg,79099.0,79160.0,NaN,79342.0,NaN,NaN,NaN,79099.0
4,435,nigel-mansell,79588.0,78785.0,NaN,77514.0,NaN,NaN,NaN,77514.0
...,...,...,...,...,...,...,...,...,...,...
14230,1125,fernando-alonso,85504.0,84574.0,84453.0,NaN,83794.0,83268.0,83196.0,83196.0
14231,1125,liam-lawson,85563.0,84503.0,84519.0,NaN,83733.0,83472.0,NaN,83472.0
14232,1125,valtteri-bottas,85611.0,84230.0,84479.0,NaN,83481.0,83341.0,83204.0,83204.0
14233,1125,guanyu-zhou,85921.0,84557.0,84668.0,NaN,83880.0,NaN,NaN,83880.0


In [8]:
pace_id_list = pacedf['raceId'].unique()

def get_pace_ALPC(input_race_id_list):
    pace_data = []

    for race_id in input_race_id_list:
        if race_id not in pacedf['raceId'].unique():
            pace_data.append({'raceId': race_id, 'paceALPC': np.nan, 'pacePosDiff': np.nan})
            continue

        this_race_data = pacedf[pacedf['raceId'] == race_id]
        this_race_start_data = start_position_data_file[start_position_data_file['raceId'] == race_id]
        this_race_start_data = this_race_start_data.sort_values('positionNumber')
        update_race_df = this_race_data.sort_values('min_time')
        driver_pace_pos = update_race_df['driverId'].to_list()
        starting_grid = this_race_start_data['driverId'].to_list()

        n = len(starting_grid)
        m = len(driver_pace_pos)
        total_sum = 0
        pos_diff = []

        if m != n: 
            common_drivers = set(driver_pace_pos) & set(starting_grid)
            driver_pace_pos = [x for x in driver_pace_pos if x in common_drivers]
            starting_grid = [y for y in starting_grid if y in common_drivers]
            n = len(starting_grid)

        for i in range(n):
            summand = 0
            for j in range(n):
                if i == j: continue
                driver_i = starting_grid[i]
                driver_j = starting_grid[j]
                local_change = (i - j) - (driver_pace_pos.index(driver_i) - driver_pace_pos.index(driver_j))
                summand += local_change

            total_sum += abs(summand) / (n*(n-1))

        for i in range(n):
            driver_i = starting_grid[i]
            pos_diff.append(abs(i - driver_pace_pos.index(driver_i)))
        
        pace_data.append({'raceId': race_id, 'paceALPC': total_sum, 'pacePosDiff': np.mean(pos_diff)})
    return pd.DataFrame(pace_data) 

get_pace_ALPC(race_id_list)

,raceId,paceALPC,pacePosDiff
0,421,NaN,NaN
1,422,NaN,NaN
2,423,NaN,NaN
3,424,NaN,NaN
4,425,NaN,NaN
...,...,...,...
700,1121,1.285714,1.200000
701,1122,4.705882,4.444444
702,1123,1.368421,1.300000
703,1124,0.210526,0.200000


In [9]:
free_df = get_free_practice_RALPC(race_id_list)
quals_df = generate_quals_ALPC_data(race_id_list)
pace_df = get_pace_ALPC(race_id_list)

dataframes = [free_df, quals_df, pace_df]
pre_alpc_df = reduce(lambda left, right: pd.merge(left, right, on='raceId', how='outer'), dataframes)
pre_alpc_df.to_csv('/home/winter-storm/f1-data-project/erdos_ds_f1/Patrick/Feature Testing Data Files/pre-race-alpc-data.csv',
                   index=False)

In [11]:
# Define columns and weights
# Use unstandardized raw values since these yielded optimized correlation
columns = ['FRALPC', 'freeAvgPosDiff', 'qualifyingALPC', 'qualAvgPosDiff', 'paceALPC', 'pacePosDiff']
weights = np.array([-0.61163616, 0.6425322, -0.02758382, 0.01785513, 0.31805315, -0.33289053])

pre_alpc_df['preRaceALPCWeightedSum'] = pre_alpc_df[columns].values @ weights
pre_alpc_df.to_csv('/home/winter-storm/f1-data-project/erdos_ds_f1/Patrick/Feature Data Files/pre race alpc data.csv',
                   index=False)